In [1]:
import sys, time, threading
sys.path.append('../')

In [2]:
from ipycanvas import canvas
from demo import *
from model import *
from pythreejs import *
from numpy import *

In [3]:
c = canvas.Canvas(width=600, height=200, sync_image_data=True)
pipe = AdvancedPipe(Circle(1, 0), Circle(1, 0), 10, canvas=c)

In [4]:
height = 7.5

In [5]:
demo = Demo(pipe.params, pipe)
cyl_obj = Cylinder(pipe.i1.d / 4, pipe.i2.d / 4, height, 16, 4)
cyl = cyl_obj.my_cyl()

In [6]:
skinIndices = []
skinWeights = []

vertices = get_attribute(cyl, 'position').array
boneHeight = height / 3
for i in range(vertices.shape[0]):

    y = vertices[i, 1] + 0.5 * height

    skinIndex = y // boneHeight
    skinWeight = ( y % boneHeight ) / boneHeight

    # Ease between each bone
    skinIndices.append([skinIndex, skinIndex + 1, 0, 0 ])
    skinWeights.append([1 - skinWeight, skinWeight, 0, 0 ])

cyl.attributes = dict(
    cyl.attributes,
    skinIndex=BufferAttribute(skinIndices),
    skinWeight=BufferAttribute(skinWeights),
)

b1 = Bone(position=(0, - height / 2, 0))
b2 = Bone(position=(0, - height / 4, 0))
b3 = Bone(position=(0, height / 4, 0))
b4 = Bone(position=(0, height / 2, 0))

b1.add(b2)
b2.add(b3)
b3.add(b4)
bones = [b1, b2, b3, b4]

skeleton = Skeleton(bones)

pipeMesh = SkinnedMesh(cyl, MeshPhongMaterial(skinning=True, color='gray', opacity=0.7, transparent=True), rotation=[pymath.pi, 0, pymath.pi / 2, 'XYZ'], position=[0, 0, -5])
pipeMesh.add(bones[0])
pipeMesh.skeleton = skeleton

In [7]:
widgets = [[] for i in range(len(bones))]
pipeMesh.skeleton.bones[0].rotation = [0, 0, -pymath.pi / 2, 'XYZ']

In [8]:
helper = SkeletonHelper(pipeMesh)

In [9]:
from ipywidgets import FloatSlider
import ipywidgets as widg
labels = [widg.Label("Bone " + str(i)) for i in range(len(bones))]
output_boxes = [[]] * len(bones)
for i in range(len(bones)):
    bone = bones[i]
    widgets[i].append(FloatSlider(description="Pos X", value=bone.position[0], min=bone.position[0]-10, max=bone.position[0] + 10))
    widgets[i].append(FloatSlider(description="Pos Y", value=bone.position[1], min=bone.position[1]-10, max=bone.position[1] + 10))
    widgets[i].append(FloatSlider(description="Pos Z", value=bone.position[2], min=bone.position[2]-10, max=bone.position[2] + 10))
    
    widgets[i].append(FloatSlider(description="Rot X", value=bone.rotation[0], min=bone.rotation[0]-10, max=bone.rotation[0] + 10))
    widgets[i].append(FloatSlider(description="Rot Y", value=bone.rotation[1], min=bone.rotation[1]-10, max=bone.rotation[1] + 10))
    widgets[i].append(FloatSlider(description="Rot Z", value=bone.rotation[2], min=bone.rotation[2]-10, max=bone.rotation[2] + 10))
    output_boxes[i] = widg.VBox([labels[i]] + widgets[i])

contr = widg.HBox(output_boxes)    

In [10]:
def on_change(args):
    for i in range(len(bones)):
        bone = bones[i]
        widget = widgets[i]
        bone.position = [widget[0].value, widget[1].value, widget[2].value]
        bone.rotation = [widget[3].value, widget[4].value, widget[5].value, 'XYZ']

In [11]:
for widget in widgets:
    for w in widget:
        w.observe(on_change)

In [12]:
def on_reset(args):
    for widget in widgets:
        for w in widget:
            w.value = w.min + 10

In [13]:
reset = widg.Button(description="Reset")
reset.on_click(on_reset)

In [14]:
#cyl = CylinderBufferGeometry(pipe.i1.d / 2, pipe.i2.d / 2, height, 16, 8)
#pipeMesh.geometry=cyl
cam = PerspectiveCamera(aspect=3.0, children=(DirectionalLight(color='white', intensity=0.5, matrixWorldNeedsUpdate=True, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), shadow=DirectionalLightShadow(camera=OrthographicCamera(bottom=-5.0, far=500.0, left=-5.0, near=0.5, position=(0.0, 0.0, 0.0), projectionMatrix=(0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, -0.004004004004004004, 0.0, 0.0, 0.0, -1.002002002002002, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), right=5.0, rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), top=5.0, up=(0.0, 1.0, 0.0)), mapSize=(512.0, 512.0)), target=Object3D(position=(0.0, 0.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), up=(0.0, 1.0, 0.0)),), position=(4.156315924607438, -1.3359333302317502, 4.78814139969309), projectionMatrix=(0.7148356401698529, 0.0, 0.0, 0.0, 0.0, 2.1445069205095586, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion=(-0.18734995211891464, -0.01811133257326921, -0.003454930848697659, 0.9821201751956282), rotation=(2.955719827270785, -0.569237255703572, 3.0405841287461377, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))

water = CylinderBufferGeometry(pipe.i1.d / 4, pipe.i1.d / 4, height / 3, 16, 8, thetaLength=pymath.pi)
waterMesh = Mesh(water, MeshStandardMaterial(color='lightblue', opacity=1, transparent=False), rotation=[pymath.pi, 0, pymath.pi / 2, 'XYZ'], position=[-height / 3, 0, -5])
scene = Scene(children=[pipeMesh, helper, waterMesh, cam, AmbientLight(color='#FFFFFF')], background=None)
renderer = Renderer(scene=scene, camera=cam, width=600, height=200, alpha=True, clearOpacity=0)
demo.show()
renderer

Output()

Output()

Renderer(camera=PerspectiveCamera(aspect=3.0, children=(DirectionalLight(color='white', intensity=0.5, matrixW…

In [15]:
def update_pipe(args):
    pipeMesh.geometry = Cylinder(pipe.i2.d / 4, pipe.i1.d / 4, height, 16, 4).my_cyl()
    
    skinIndices = []
    skinWeights = []

    vertices = get_attribute(pipeMesh.geometry, 'position').array
    boneHeight = height / 3
    for i in range(vertices.shape[0]):

        y = vertices[i, 1] + 0.5 * height

        skinIndex = y // boneHeight
        skinWeight = ( y % boneHeight ) / boneHeight

        # Ease between each bone
        skinIndices.append([skinIndex, skinIndex + 1, 0, 0 ])
        skinWeights.append([1 - skinWeight, skinWeight, 0, 0 ])

    pipeMesh.geometry.attributes = dict(
        pipeMesh.geometry.attributes,
        skinIndex=BufferAttribute(skinIndices),
        skinWeight=BufferAttribute(skinWeights),
    )

    b1 = Bone(position=(0, - height / 2, 0))
    b2 = Bone(position=(0, - height / 4, 0))
    b3 = Bone(position=(0, height / 4, 0))
    b4 = Bone(position=(0, height / 2, 0))

    b1.add(b2)
    b2.add(b3)
    b3.add(b4)
    
    global bones
    pipeMesh.remove(bones[0])
    
    bones = [b1, b2, b3, b4]

    skeleton = Skeleton(bones)

    # pipeMesh = SkinnedMesh(cyl, MeshPhongMaterial(skinning=True, color='gray', opacity=0.7, transparent=True), rotation=[pymath.pi, 0, pymath.pi / 2, 'XYZ'], position=[0, 0, -5])
    pipeMesh.add(bones[0])
    pipeMesh.skeleton = skeleton
    pipeMesh.skeleton.bones[0].rotation = [0, 0, -pymath.pi / 2, 'XYZ']
    
    pipeMesh.skeleton.bones[2].position = [(pipe.i1yParam.real() - pipe.i2yParam.real()) / 25, pipeMesh.skeleton.bones[2].position[1], pipeMesh.skeleton.bones[2].position[2]]
    #pipeMesh.geometry.exec_three_obj_method("dispose")
    #pipeMesh.geometry = CylinderBufferGeometry(pipe.i1.d / 2, pipe.i2.d / 2, height, 16, 8)
    waterMesh.rotation = [waterMesh.rotation[0], waterMesh.rotation[1], pipeMesh.skeleton.bones[1].rotation[2], 'XYZ']
    g3 = LineSegmentsGeometry(
    positions=[
        [[0, -2.5 + (pipe.i1yParam.real() - pipe.i2yParam.real()) / 25, -5], [100, -2.5 + (pipe.i1yParam.real() - pipe.i2yParam.real()) / 25, -5]],
        [[0, 0, -5], [100, 0, -5]]
    ],
    colors=[
        [[1, 0, 0], [1, 0, 1]],
        [[1, 0, 0], [1, 0, 1]]
    ],
    )
    m3 = LineMaterial(linewidth=10, vertexColors='VertexColors')
    line3 = LineSegments2(g3, m3)
    scene.add(line3)
    #scene.add(pipeMesh)
    #scene.add(line3)
    #pipeMesh.skeleton.bones[2].rotation = [0, 0, pipe.i2yParam.real() / 100, 'XYZ']
    # cyl = CylinderBufferGeometry(pipe.i1.d / 2, pipe.i2.d / 2, height, 16, 8)
    #cyl_geom = BufferGeometry.from_geometry(cyl)
    #vertices = cylinder.attributes['position'].array
    #vertices[15 // 2] = [0, 0, 0]
    #print(cylGeom.vertices)
    # pipeMesh.geometry = cyl
    
    #cv = pipe.get_drawing()
    #scene.add(cv)
    
    #pipeMesh.position = [pipeMesh.position[0], pipe.i1yParam.real(), pipeMesh.position[2]]

In [16]:
pipe.observe(update_pipe)